In [1]:
import sys
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from collections import Counter

sys.path.append("../../.")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms import num_comparison, nat_numbers, arithm_ops, combinators, pairs, logic
from calculus_path_mod.terms.pseudonym import *

from calculus_path_mod.json_serialization import load_terms

# Load Terms filtered by LO & RI

In [2]:
lists_terms_LO = [load_terms(f"../../tests_11_retests/collected_terms/terms_210_filtered_LO_{inx_}.dat") for inx_ in range(20)]

# Collect more terms with normalization process data for LO & LI strategies with terms_LO & terms_RI

In [3]:
def gen_norm_data(terms_list, strategy):
    normalized_terms_dict = dict()
    for term in tqdm(terms_list):
        term_name = term.simple_str()
        normalized_terms_dict[term_name] = []
        term_red_steps = 0
        (step_term, _, _), norm_term = term.one_step_normalize_visual(strategy)
        normalized_terms_dict[term_name].append(step_term.simple_str())

        while norm_term:
            normalized_terms_dict[term_name].append(norm_term.simple_str())
            (step_term, _, _), norm_term = norm_term.one_step_normalize_visual(strategy)

            # computation limitation
            if (step_term.vertices_number > 3_000) or (term_red_steps > 400):
                norm_term = None
    return normalized_terms_dict

In [4]:
list_res_OO = [gen_norm_data(terms_LO, RIStrategy()) for terms_LO in lists_terms_LO]

 83%|████████▎ | 192/231 [48:52<09:55, 15.28s/it]

KeyboardInterrupt



# Prepare the dataset

In [ ]:
steps_lo = []
simple_terms = []

for res_ in list_res_OO:
    for key_ in res_.keys():
        list_red_steps = res_[key_]
        total_steps = len(list_red_steps) - 1
        for inx_ in range(total_steps + 1):
            if list_red_steps[inx_] not in simple_terms:
                simple_terms.append(list_red_steps[inx_])
                steps_lo.append(total_steps - inx_)

In [ ]:
df = pd.DataFrame({"steps_num": steps_lo, "simple_terms": simple_terms})
print(len(df))
df = df.drop_duplicates(subset="simple_terms")
print(len(df))

In [ ]:
len(set(df["simple_terms"]))

In [ ]:
df.to_csv("./data_RI/steps_simple_term_str_extended.csv", index=False)

In [2]:
all_data = pd.read_csv("data_RI/steps_simple_term_str_extended_v1_0_2.csv", delimiter=',')
all_data = pd.concat([all_data, pd.read_csv("data_RI/steps_simple_term_str_extended_v1_3_5.csv", delimiter=',')])
all_data = pd.concat([all_data, pd.read_csv("data_RI/steps_simple_term_str_extended_v1_6_8.csv", delimiter=',')])
all_data = pd.concat([all_data, pd.read_csv("data_RI/steps_simple_term_str_extended_v1_9_11.csv", delimiter=',')])
all_data = pd.concat([all_data, pd.read_csv("data_RI/steps_simple_term_str_extended_v1_12_14.csv", delimiter=',')])
all_data = pd.concat([all_data, pd.read_csv("data_RI/steps_simple_term_str_extended_v1_15_17.csv", delimiter=',')])
all_data = pd.concat([all_data, pd.read_csv("data_RI/steps_simple_term_str_extended_v1_18_19.csv", delimiter=',')])

In [3]:
print(len(all_data))
all_data = all_data.drop_duplicates(subset="simple_terms")
print(len(all_data))

63632
62347


In [4]:
all_data.to_csv("data_RI/steps_simple_term_str_extended.csv")